### 1. Preliminray

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from itertools import product
from solver import train_resnet_with_cub
from solver import test_resnet_with_cub

DATA_PATH = os.path.join("data", "CUB_200_2011")

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

### 2. Grid-Search Optimal Learning Rate

#### 2.1 Set the range of hyper-parameters 

In [2]:
# set hyper-parameters here
num_epochs = [15]
fine_tuning_lrs = [5e-5, 1e-4, 5e-4]
output_lrs = [1e-3, 2e-3, 5e-3, 8e-3, 1e-2]

configurations = list(product(fine_tuning_lrs, output_lrs))

#### 2.2 Train

In [6]:
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(num_epochs, fine_tuning_lr=config[0], output_lr=config[1])
    best_accs.extend(curr_best_acc)

best_acc = max(best_accs)    
best_fine_tune_lr, best_output_lr = None, None

# write the results into a txt file
with open('best_accuracy_lr.txt', 'w') as f:
    f.write("Configuration        Accuracy\n")
    f.write("=" * 30 + "\n")
    for config, accuracy in zip(configurations, best_accs):
        # find the best config
        if accuracy == best_acc and best_fine_tune_lr is None:
            best_fine_tune_lr, best_output_lr = config
        # write the training record into file 
        f.write("({:>7.5f}, {:>7.5f})   {:>8.6f}\n".format(config[0], config[1], accuracy))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 138MB/s]


Training with configuration (0.00005, 0.00100)
[Epoch  1 / 15], Training loss is 6.988053
[Epoch  1 / 15], Validation loss is 6.522593, Top-5 accuracy is 0.035899, Top-1 accuracy is 0.007767
[Epoch  2 / 15], Training loss is 5.983014
[Epoch  2 / 15], Validation loss is 5.964954, Top-5 accuracy is 0.057473, Top-1 accuracy is 0.013635
[Epoch  3 / 15], Training loss is 5.492842
[Epoch  3 / 15], Validation loss is 5.580335, Top-5 accuracy is 0.089748, Top-1 accuracy is 0.023127
[Epoch  4 / 15], Training loss is 5.079500
[Epoch  4 / 15], Validation loss is 5.219078, Top-5 accuracy is 0.133241, Top-1 accuracy is 0.037107
[Epoch  5 / 15], Training loss is 4.715111
[Epoch  5 / 15], Validation loss is 4.890339, Top-5 accuracy is 0.182085, Top-1 accuracy is 0.057818
[Epoch  6 / 15], Training loss is 4.351164
[Epoch  6 / 15], Validation loss is 4.560553, Top-5 accuracy is 0.244046, Top-1 accuracy is 0.085951
[Epoch  7 / 15], Training loss is 4.019405
[Epoch  7 / 15], Validation loss is 4.265923, 

#### 2.3 Inspect the Accuracy

In [1]:
!more .\Output\best_accuracy_lr.txt

Configuration        Accuracy
(0.00005, 0.00100)   0.321540
(0.00005, 0.00200)   0.429928
(0.00005, 0.00500)   0.519330
(0.00005, 0.00800)   0.542630
(0.00005, 0.01000)   0.547463
(0.00010, 0.00100)   0.427684
(0.00010, 0.00200)   0.487401
(0.00010, 0.00500)   0.553158
(0.00010, 0.00800)   0.569900
(0.00010, 0.01000)   0.575078
(0.00050, 0.00100)   0.570590
(0.00050, 0.00200)   0.583362
(0.00050, 0.00500)   0.608215
(0.00050, 0.00800)   0.618053
(0.00050, 0.01000)   0.619261


### 3. Find the Optimal Training Epoch

#### 3.1 Train on a larger epoch

In [8]:
num_epochs = [5 * (i + 1) for i in range(9)]

# init the configurations
best_accs = []

# train with the pretrained model
curr_best_acc = train_resnet_with_cub(num_epochs, best_fine_tune_lr, best_output_lr, save=True)
best_accs.extend(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy_ep.txt', 'w') as f:
    f.write("Epoch  Accuracy\n")
    f.write("=" * 50 + "\n")
    for num_epoch, accuracy in zip(num_epochs, best_accs):
        f.write("{:>2}     {:>8.6f}\n".format(num_epoch, accuracy))

Training with configuration (0.00050, 0.01000)
[Epoch  1 / 45], Training loss is 5.481130
[Epoch  1 / 45], Validation loss is 3.924461, Top-5 accuracy is 0.377977, Top-1 accuracy is 0.157577
[Epoch  2 / 45], Training loss is 2.759579
[Epoch  2 / 45], Validation loss is 2.549399, Top-5 accuracy is 0.679323, Top-1 accuracy is 0.372627
[Epoch  3 / 45], Training loss is 1.783293
[Epoch  3 / 45], Validation loss is 2.045697, Top-5 accuracy is 0.774422, Top-1 accuracy is 0.475147
[Epoch  4 / 45], Training loss is 1.310192
[Epoch  4 / 45], Validation loss is 1.847726, Top-5 accuracy is 0.812392, Top-1 accuracy is 0.511909
[Epoch  5 / 45], Training loss is 1.016611
[Epoch  5 / 45], Validation loss is 1.706180, Top-5 accuracy is 0.834311, Top-1 accuracy is 0.545737
[Epoch  6 / 45], Training loss is 0.810399
[Epoch  6 / 45], Validation loss is 1.621288, Top-5 accuracy is 0.847946, Top-1 accuracy is 0.567656
[Epoch  7 / 45], Training loss is 0.657625
[Epoch  7 / 45], Validation loss is 1.565100, 

#### 3.2 Inspect the Accuracy

In [1]:
!more .\Output\best_accuracy_ep.txt

Epoch  Accuracy
 5     0.545737
10     0.608043
15     0.619261
20     0.637038
25     0.639455
30     0.642734
35     0.647912
40     0.647912
45     0.647912


### 4. Train with Random Initialized Weights

In [9]:
curr_best_acc = train_resnet_with_cub(num_epochs, best_fine_tune_lr, best_output_lr, pretrain=False)

Training with configuration (0.00050, 0.01000)
[Epoch  1 / 45], Training loss is 5.624804
[Epoch  1 / 45], Validation loss is 5.383454, Top-5 accuracy is 0.045047, Top-1 accuracy is 0.009665
[Epoch  2 / 45], Training loss is 5.279906
[Epoch  2 / 45], Validation loss is 5.273602, Top-5 accuracy is 0.059372, Top-1 accuracy is 0.011046
[Epoch  3 / 45], Training loss is 5.127752
[Epoch  3 / 45], Validation loss is 5.121473, Top-5 accuracy is 0.085088, Top-1 accuracy is 0.021056
[Epoch  4 / 45], Training loss is 4.961903
[Epoch  4 / 45], Validation loss is 5.093522, Top-5 accuracy is 0.111495, Top-1 accuracy is 0.030031
[Epoch  5 / 45], Training loss is 4.834755
[Epoch  5 / 45], Validation loss is 4.984315, Top-5 accuracy is 0.114946, Top-1 accuracy is 0.032102
[Epoch  6 / 45], Training loss is 4.685044
[Epoch  6 / 45], Validation loss is 4.907023, Top-5 accuracy is 0.133586, Top-1 accuracy is 0.034691
[Epoch  7 / 45], Training loss is 4.592460
[Epoch  7 / 45], Validation loss is 4.875474, 

### 5. Test the trained model

In [7]:
MODEL_PATH = os.path.join("Output", "resnet18_cub.pth")
test_resnet_with_cub(DATA_PATH, MODEL_PATH)

100%|██████████| 91/91 [02:40<00:00,  1.77s/it]

For the class     Black_footed_Albatross     on the CUB dataset, Top-1 accuracy is 0.666667, Top-5 accuracy is 0.933333
For the class        Laysan_Albatross        on the CUB dataset, Top-1 accuracy is 0.600000, Top-5 accuracy is 0.800000
For the class        Sooty_Albatross         on the CUB dataset, Top-1 accuracy is 0.547297, Top-5 accuracy is 1.000000
For the class       Groove_billed_Ani        on the CUB dataset, Top-1 accuracy is 0.600000, Top-5 accuracy is 0.966667
For the class         Crested_Auklet         on the CUB dataset, Top-1 accuracy is 0.793103, Top-5 accuracy is 0.913793
For the class          Least_Auklet          on the CUB dataset, Top-1 accuracy is 0.909091, Top-5 accuracy is 1.000000
For the class        Parakeet_Auklet         on the CUB dataset, Top-1 accuracy is 0.782609, Top-5 accuracy is 0.956522
For the class       Rhinoceros_Auklet        on the CUB dataset, Top-1 accuracy is 0.611111, Top-5 accuracy is 0.888889
For the class        Brewer_Blackbird   